# Simplify and Scale Data Engineering Pipelines with Delta Lake

Delta Lake: An open-source storage format that brings ACID transactions to Apache Spark™ and big data workloads.

<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>


* **Open format**: Stored as Parquet format in blob storage.
* **ACID Transactions**: Ensures data integrity and read consistency with complex, concurrent data pipelines.
* **Schema Enforcement and Evolution**: Ensures data cleanliness by blocking writes with unexpected.
* **Audit History**: History of all the operations that happened in the table.
* **Time Travel**: Query previous versions of the table by time or version number.
* **Deletes and upserts**: Supports deleting and upserting into tables with programmatic APIs.
* **Scalable Metadata management**: Able to handle millions of files are scaling the metadata operations with Spark.
* **Unified Batch and Streaming Source and Sink**: A table in Delta Lake is both a batch table, as well as a streaming source and sink. Streaming data ingest, batch historic backfill, and interactive queries all just work out of the box. 

### Source:
This notebook is a modified version of the [SAIS EU 2019 Delta Lake Tutorial](https://github.com/delta-io/delta/tree/master/examples/tutorials/saiseu19).

### Steps to run this notebook

You can run this notebook in a Databricks environment. Specifically, this notebook has been designed to run in [Databricks Community Edition](http://community.cloud.databricks.com/) as well.
To run this notebook, you have to [create a cluster](https://docs.databricks.com/clusters/create.html) with version **Databricks Runtime 6.1 or later** and [attach this notebook](https://docs.databricks.com/notebooks/notebooks-manage.html#attach-a-notebook-to-a-cluster) to that cluster. <br/>&nbsp;

### Source Data for this notebook

The data used is a modified version of the public data from [Lending Club](https://www.kaggle.com/wendykan/lending-club-loan-data). It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).

## Explore data as a Parquet table
* Initially, let's start by exploring the data as a Parquet table.  
* As we progress, we will showcase how Delta Lake improves up on Parquet.

#### Download the sampled Lending Club data

In [5]:
%sh rm -rf /dbfs/tmp/sais_eu_19_demo/ && mkdir -p /dbfs/tmp/sais_eu_19_demo/loans/ && wget -O /dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet  https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet && ls -al  /dbfs/tmp/sais_eu_19_demo/loans/ 

--2020-03-12 16:46:21-- https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet
Resolving pages.databricks.com (pages.databricks.com)... 104.17.74.206, 104.17.70.206, 104.17.71.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.74.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164631 (161K) [text/plain]
Saving to: ‘/dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet’

 0K .......... .......... .......... .......... .......... 31% 5.04M 0s
 50K .......... .......... .......... .......... .......... 62% 12.6M 0s
 100K .......... .......... .......... .......... .......... 93% 8.74M 0s
 150K .......... 100% 17.5M=0.02s

2020-03-12 16:46:21 (7.96 MB/s) - ‘/dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet’ saved [164631/164631]

total 169
drwxrwxrwx 2 root root 4096 Mar 12 2020 .
drwxrwxrwx 2 root root 4096 Mar 12 16:46 ..
-rwxrwxrwx 1 root root 164631 Mar 12 2020 SAISEU19-loan-risks.snappy.parquet

#### Create the parquet table "loans_parquet"

In [7]:
from pyspark.sql.functions import * 

parquet_path = "/tmp/sais_eu_19_demo/loans/"

# Create a view on the table called loans_parquet
spark.read.format("parquet").load(parquet_path).createOrReplaceTempView("loans_parquet")
print("Defined view 'loans_parquet'")

Defined view 'loans_parquet'

#### Let's explore this parquet table.

*Schema of the table*

| Column Name | Description |
| ----------- | ----------- | 
| load_id | unique id for each loan |
| funded_amnt | principal amount of the loan funded to the loanee |
| paid_amnt | amount from the principle that has been paid back (ignoring interests) |
| addr_state | state where this loan was funded |

In [9]:
spark.sql("select * from loans_parquet").show(20)

+-------+-----------+---------+----------+
loan_id|funded_amnt|paid_amnt|addr_state|
+-------+-----------+---------+----------+
 0| 1000| 182.22| CA|
 1| 1000| 361.19| WA|
 2| 1000| 176.26| TX|
 3| 1000| 1000.0| OK|
 4| 1000| 249.98| PA|
 5| 1000| 408.6| CA|
 6| 1000| 1000.0| MD|
 7| 1000| 168.81| OH|
 8| 1000| 193.64| TX|
 9| 1000| 218.83| CT|
 10| 1000| 322.37| NJ|
 11| 1000| 400.61| NY|
 12| 1000| 1000.0| FL|
 13| 1000| 165.88| NJ|
 14| 1000| 190.6| TX|
 15| 1000| 1000.0| OH|
 16| 1000| 213.72| MI|
 17| 1000| 188.89| MI|
 18| 1000| 237.41| CA|
 19| 1000| 203.85| CA|
+-------+-----------+---------+----------+
only showing top 20 rows

#### How many records does it have?

In [11]:
spark.sql("select count(*) from loans_parquet").show()

+--------+
count(1)|
+--------+
 14705|
+--------+

In [12]:
dbutils.notebook.exit("stop") # Stop the notebook before the streaming cell, in case of a "run all" 

stop

#### Let's start appending some new data to it using Structured Streaming

We will generate a stream of data from with randomly generated loan ids and amounts. 
In addition, we are going to define a few more useful utility functions.

In [14]:
import random
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *


def random_checkpoint_dir(): 
  return "/tmp/sais_eu_19_demo/chkpt/%s" % str(random.randint(0, 10000))

# User-defined function to generate random state

states = ["CA", "TX", "NY", "WA"]

@udf(returnType=StringType())
def random_state():
  return str(random.choice(states))

# Function to start a streaming query with a stream of randomly generated data and append to the parquet table
def generate_and_append_data_stream(table_format, table_path):

  stream_data = (spark.readStream.format("rate").option("rowsPerSecond", 5).load() 
    .withColumn("loan_id", 10000 + col("value")) 
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer")) 
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000)) 
    .withColumn("addr_state", random_state()))

  query = (stream_data.writeStream 
    .format(table_format) 
    .option("checkpointLocation", random_checkpoint_dir()) 
    .trigger(processingTime = "10 seconds") 
    .start(table_path))

  return query

# Function to stop all streaming queries 
def stop_all_streams():
  # Stop all the streams
  print("Stopping all streams")
  for s in spark.streams.active:
    s.stop()
  print("Stopped all streams")
  print("Deleting checkpoints")  
  dbutils.fs.rm("/tmp/sais_eu_19_demo/chkpt/", True)
  print("Deleted checkpoints")

#### Let's start a new stream to append data to the Parquet table

In [16]:
stream_query = generate_and_append_data_stream(
    table_format = "parquet", 
    table_path = parquet_path)

#### Let's see if the data is being added to the table or not

In [18]:
spark.read.format("parquet").load(parquet_path).count()

Out[16]: 870

#### What happens if we try to add a second writeStream?

In [20]:
stream_query2 = generate_and_append_data_stream(
    table_format = "parquet", 
    table_path = parquet_path)

#### Where did our existing 14705 rows go? Let's see the data once again

In [22]:
spark.read.format("parquet").load(parquet_path).show() # wrong schema!

+--------------------+-----+-------+-----------+------------------+----------+
 timestamp|value|loan_id|funded_amnt| paid_amnt|addr_state|
+--------------------+-----+-------+-----------+------------------+----------+
2020-03-12 17:30:...| 721| 10721| 8515| 7708.564399507631| TX|
2020-03-12 17:30:...| 737| 10737| 8483| 6867.059786862704| NY|
2020-03-12 17:30:...| 753| 10753| 6373| 4411.101043842316| WA|
2020-03-12 17:30:...| 769| 10769| 6863| 4999.44541383953| WA|
2020-03-12 17:28:...| 170| 10170| 7894| 6668.754596616619| CA|
2020-03-12 17:29:...| 186| 10186| 5915| 4393.331127795706| TX|
2020-03-12 17:29:...| 202| 10202| 7486| 6189.330254905916| TX|
2020-03-12 17:29:...| 218| 10218| 9338| 8002.369402129733| NY|
2020-03-12 17:29:...| 471| 10471| 5270| 4293.177224348021| WA|
2020-03-12 17:30:...| 487| 10487| 7988| 7849.831605533605| CA|
2020-03-12 17:30:...| 503| 10503| 6605| 4857.451745434359| WA|
2020-03-12 17:30:...| 519| 10519| 7932| 7511.839312598428| NY|
2020-03-12 17:29:...| 221| 10221| 7762| 6751.638423745441| WA|
2020-03-12 17:29:...| 237| 10237| 5067|3330.9785637563455| CA|
2020-03-12 17:29:...| 253| 10253| 8314| 6740.187582158214| NY|
2020-03-12 17:29:...| 269| 10269| 7668| 6100.75465788365| WA|
2020-03-12 17:30:...| 671| 10671| 6505| 5732.536576112458| CA|
2020-03-12 17:30:...| 687| 10687| 6272| 6137.680516266621| NY|
2020-03-12 17:30:...| 703| 10703| 6451| 4645.2681010385| NY|
2020-03-12 17:30:...| 719| 10719| 5962| 4553.73606646309| CA|
+--------------------+-----+-------+-----------+------------------+----------+
only showing top 20 rows

#### Where did the two new columns `timestamp` and `value` come from? What happened here!

What really happened is that when the streaming query started adding new data to the Parquet table, it did not properly account for the existing data in the table. Furthermore, the new data files that written out accidentally had two extra columns in the schema. Hence, when reading the table, the 2 different schema from different files were merged together, thus unexpectedly modifying the schema of the table.


Before we move on, **if you are running on Databricks Community Edition, definitely stop the streaming queries.** 

You free account in Databricks Community Edition has quota limits on the number of files and we do not want to hit that quote limit by running the streaming queries for too long.

In [24]:
stop_all_streams()

Stopping all streams
Stopped all streams
Deleting checkpoints
Deleted checkpoints

### Problems with Parquet format

Parquet is only a data layout format within a single file, does not provide any guarantees across an entire table of many parquet files.

#### 1. No schema enforcement 
Schema is not enforced when writing leading to dirty and often corrupted data.

#### 2. No interoperatbility between batch and streaming workloads
Apache Spark's Parquet streaming sink does not maintain enough metadata such that batch workload can seamlessly interact with batch workloads.

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Batch + stream processing and schema enforcement with Delta Lake
Let's understand Delta Lake solves these particular problems (among many others). We will start by creating a Delta table from the original data.

In [27]:
%sh rm -rf /dbfs/tmp/sais_eu_19_demo/ && mkdir -p /dbfs/tmp/sais_eu_19_demo/loans/ && wget -O /dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet  https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet && ls -al  /dbfs/tmp/sais_eu_19_demo/loans/ && ls -al  /dbfs/tmp/sais_eu_19_demo/

--2020-03-12 17:34:30-- https://pages.databricks.com/rs/094-YMS-629/images/SAISEU19-loan-risks.snappy.parquet
Resolving pages.databricks.com (pages.databricks.com)... 104.17.70.206, 104.17.71.206, 104.17.72.206, ...
Connecting to pages.databricks.com (pages.databricks.com)|104.17.70.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 164631 (161K) [text/plain]
Saving to: ‘/dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet’

 0K .......... .......... .......... .......... .......... 31% 4.13M 0s
 50K .......... .......... .......... .......... .......... 62% 8.15M 0s
 100K .......... .......... .......... .......... .......... 93% 13.0M 0s
 150K .......... 100% 17.2M=0.02s

2020-03-12 17:34:30 (7.07 MB/s) - ‘/dbfs/tmp/sais_eu_19_demo/loans/SAISEU19-loan-risks.snappy.parquet’ saved [164631/164631]

total 169
drwxrwxrwx 2 root root 4096 Mar 12 2020 .
drwxrwxrwx 2 root root 4096 Mar 12 17:34 ..
-rwxrwxrwx 1 root root 164631 Mar 12 2020 SAISEU19-loan-risks.snappy.parquet
total 12
drwxrwxrwx 2 root root 4096 Mar 12 2020 .
drwxrwxrwx 2 root root 4096 Mar 12 17:34 ..
drwxrwxrwx 2 root root 4096 Mar 12 2020 loans

In [28]:
spark.sql("set spark.sql.shuffle.partitions = 1")
spark.sql("set spark.databricks.delta.snapshotPartitions = 1")

# Configure Delta Lake Silver Path
delta_path = "/tmp/sais_eu_19_demo/loans_delta"

# Configurations necessary for running of Databricks Community Edition
spark.sql("set spark.sql.shuffle.partitions = 1")
spark.sql("set spark.databricks.delta.snapshotPartitions = 1")

# Remove folder if it exists
print("Deleting directory " + delta_path)
dbutils.fs.rm(delta_path, recurse=True)

# Create the Delta table with the same loans data
spark.read.format("parquet").load(parquet_path) \
  .write.format("delta").save(delta_path)
print("Created a Delta table at " + delta_path)

spark.read.format("delta").load(delta_path).createOrReplaceTempView("loans_delta")
print("Defined view 'loans_delta'")


Deleting directory /tmp/sais_eu_19_demo/loans_delta
Created a Delta table at /tmp/sais_eu_19_demo/loans_delta
Defined view 'loans_delta'

#### Let's see the data once again

In [30]:
spark.sql("select count(*) from loans_delta").show()

+--------+
count(1)|
+--------+
 14705|
+--------+

In [31]:
spark.sql("select * from loans_delta").show()

+-------+-----------+---------+----------+
loan_id|funded_amnt|paid_amnt|addr_state|
+-------+-----------+---------+----------+
 0| 1000| 182.22| CA|
 1| 1000| 361.19| WA|
 2| 1000| 176.26| TX|
 3| 1000| 1000.0| OK|
 4| 1000| 249.98| PA|
 5| 1000| 408.6| CA|
 6| 1000| 1000.0| MD|
 7| 1000| 168.81| OH|
 8| 1000| 193.64| TX|
 9| 1000| 218.83| CT|
 10| 1000| 322.37| NJ|
 11| 1000| 400.61| NY|
 12| 1000| 1000.0| FL|
 13| 1000| 165.88| NJ|
 14| 1000| 190.6| TX|
 15| 1000| 1000.0| OH|
 16| 1000| 213.72| MI|
 17| 1000| 188.89| MI|
 18| 1000| 237.41| CA|
 19| 1000| 203.85| CA|
+-------+-----------+---------+----------+
only showing top 20 rows

#### Let's run a streaming count(*) on the table so that the count updates automatically

In [33]:
spark.readStream.format("delta").load(delta_path).createOrReplaceTempView("loans_delta_stream")
display(spark.sql("select count(*) from loans_delta_stream"))

count(1)
27455


#### Now let's try writing the streaming appends once again

In [35]:
stream_query_2 = generate_and_append_data_stream(table_format = "delta", table_path = delta_path)

The writes were blocked because the schema of the new data did not match the schema of table (see the exception details). See more information about how it works [here](https://databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html).

**Now, let's fix the streaming query by selecting the columns we want to write.**

In [37]:
from pyspark.sql.functions import *

# Generate a stream of randomly generated load data and append to the parquet table
def generate_and_append_data_stream_fixed(table_format, table_path):
    
  stream_data = (spark.readStream.format("rate").option("rowsPerSecond", 50).load() 
    .withColumn("loan_id", 10000 + col("value")) 
    .withColumn("funded_amnt", (rand() * 5000 + 5000).cast("integer")) 
    .withColumn("paid_amnt", col("funded_amnt") - (rand() * 2000)) 
    .withColumn("addr_state", random_state()) 
    .select("loan_id", "funded_amnt", "paid_amnt", "addr_state")   # *********** FIXED THE SCHEMA OF THE GENERATED DATA *************
    )

  query = (stream_data.writeStream 
    .format(table_format) 
    .option("checkpointLocation", random_checkpoint_dir()) 
    .trigger(processingTime="10 seconds") 
    .start(table_path))

  return query

#### Now we can successfully write to the table. Note the count in the above streaming query increasing as we write to this table.

In [39]:
stream_query_2 = generate_and_append_data_stream_fixed(table_format = "delta", table_path = delta_path)

**Scroll back up to see the numbers change in the `readStream` as more data is being appended by the `writeStream`.** 

**In fact, we can run multiple concurrent streams writing to that table, it will work together.**

In [41]:
stream_query_3 = generate_and_append_data_stream_fixed(table_format = "delta", table_path = delta_path)

Just for sanity check, let's query as a batch

Note, you can run a read stream, two write streams, and read in batch - concurrently!

In [43]:
spark.sql("select count(*) from loans_delta").show()

+--------+
count(1)|
+--------+
 23455|
+--------+

#### Again, remember to stop all the streaming queries.

In [45]:
stop_all_streams()

Stopping all streams
Stopped all streams
Deleting checkpoints
Deleted checkpoints

#### Let's take a look at the file system

In [47]:
%fs ls /tmp/sais_eu_19_demo/loans_delta

path,name,size
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/,_delta_log/,0
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-03fa53af-ea62-4990-883c-6806f29bcff1-c000.snappy.parquet,part-00000-03fa53af-ea62-4990-883c-6806f29bcff1-c000.snappy.parquet,164673
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-1661aaca-96df-4655-97b0-ce2926578941-c000.snappy.parquet,part-00000-1661aaca-96df-4655-97b0-ce2926578941-c000.snappy.parquet,1728
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-29d2cacc-a450-4004-beb5-a253ee37623f-c000.snappy.parquet,part-00000-29d2cacc-a450-4004-beb5-a253ee37623f-c000.snappy.parquet,1726
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-390eb5c1-711d-4435-b3d8-3f765721fd3e-c000.snappy.parquet,part-00000-390eb5c1-711d-4435-b3d8-3f765721fd3e-c000.snappy.parquet,1719
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-3e842c21-b524-459e-8489-277378550029-c000.snappy.parquet,part-00000-3e842c21-b524-459e-8489-277378550029-c000.snappy.parquet,1724
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-4722b3f4-3bd3-4991-807a-cb66a843a474-c000.snappy.parquet,part-00000-4722b3f4-3bd3-4991-807a-cb66a843a474-c000.snappy.parquet,554
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-4e783ae9-5ffc-4c88-85bd-368d65223588-c000.snappy.parquet,part-00000-4e783ae9-5ffc-4c88-85bd-368d65223588-c000.snappy.parquet,1725
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-4ed53315-4a76-4870-ac85-1fa47ce1cf35-c000.snappy.parquet,part-00000-4ed53315-4a76-4870-ac85-1fa47ce1cf35-c000.snappy.parquet,1724
dbfs:/tmp/sais_eu_19_demo/loans_delta/part-00000-4fd1a90c-0f9f-4f5a-9574-fcd71c0be639-c000.snappy.parquet,part-00000-4fd1a90c-0f9f-4f5a-9574-fcd71c0be639-c000.snappy.parquet,1729


In [48]:
%fs ls /tmp/sais_eu_19_demo/loans_delta/_delta_log/

path,name,size
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/.s3-optimization-0,.s3-optimization-0,0
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/.s3-optimization-1,.s3-optimization-1,0
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/.s3-optimization-2,.s3-optimization-2,0
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,91
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,1396
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,91
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,731
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000002.crc,00000000000000000002.crc,92
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000002.json,00000000000000000002.json,8480
dbfs:/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000003.crc,00000000000000000003.crc,92


In [49]:
%sh
head /dbfs/tmp/sais_eu_19_demo/loans_delta/_delta_log/00000000000000000026.json

{"commitInfo":{"timestamp":1584034841158,"userId":"100599","userName":"denny.lee@databricks.com","operation":"STREAMING UPDATE","operationParameters":{"outputMode":"Append","queryId":"05928668-07f5-4bde-bd41-b7fe48662d5f","epochId":"14"},"notebook":{"notebookId":"5918501"},"clusterId":"0127-045215-pined152","readVersion":24,"isolationLevel":"WriteSerializable","isBlindAppend":true}}
{"txn":{"appId":"05928668-07f5-4bde-bd41-b7fe48662d5f","version":14,"lastUpdated":1584034841157}}
{"add":{"path":"part-00000-1661aaca-96df-4655-97b0-ce2926578941-c000.snappy.parquet","partitionValues":{},"size":1728,"modificationTime":1584034841000,"dataChange":true,"stats":"{\"numRecords\":32,\"minValues\":{\"loan_id\":16050,\"funded_amnt\":5065,\"paid_amnt\":3904.9994167964064,\"addr_state\":\"CA\"},\"maxValues\":{\"loan_id\":16546,\"funded_amnt\":9985,\"paid_amnt\":9496.766314862003,\"addr_state\":\"WA\"},\"nullCount\":{\"loan_id\":0,\"funded_amnt\":0,\"paid_amnt\":0,\"addr_state\":0}}"}}
{"add":{"path":"part-00001-486a6c2c-6eb8-4f07-aafd-3618a6e28622-c000.snappy.parquet","partitionValues":{},"size":1725,"modificationTime":1584034841000,"dataChange":true,"stats":"{\"numRecords\":32,\"minValues\":{\"loan_id\":16051,\"funded_amnt\":5063,\"paid_amnt\":3484.821776571834,\"addr_state\":\"CA\"},\"maxValues\":{\"loan_id\":16547,\"funded_amnt\":9376,\"paid_amnt\":9090.237979195676,\"addr_state\":\"WA\"},\"nullCount\":{\"loan_id\":0,\"funded_amnt\":0,\"paid_amnt\":0,\"addr_state\":0}}"}}
{"add":{"path":"part-00002-08b7b308-2940-48d9-afdb-91ff47a471b2-c000.snappy.parquet","partitionValues":{},"size":1725,"modificationTime":1584034841000,"dataChange":true,"stats":"{\"numRecords\":32,\"minValues\":{\"loan_id\":16052,\"funded_amnt\":5199,\"paid_amnt\":4125.258436277146,\"addr_state\":\"CA\"},\"maxValues\":{\"loan_id\":16548,\"funded_amnt\":9468,\"paid_amnt\":8947.94838831493,\"addr_state\":\"WA\"},\"nullCount\":{\"loan_id\":0,\"funded_amnt\":0,\"paid_amnt\":0,\"addr_state\":0}}"}}
{"add":{"path":"part-00003-b4a01ca3-d95c-49ac-9e90-fdaf8307a424-c000.snappy.parquet","partitionValues":{},"size":1725,"modificationTime":1584034841000,"dataChange":true,"stats":"{\"numRecords\":32,\"minValues\":{\"loan_id\":16053,\"funded_amnt\":5050,\"paid_amnt\":3291.45752157318,\"addr_state\":\"CA\"},\"maxValues\":{\"loan_id\":16549,\"funded_amnt\":9884,\"paid_amnt\":9125.757661936086,\"addr_state\":\"WA\"},\"nullCount\":{\"loan_id\":0,\"funded_amnt\":0,\"paid_amnt\":0,\"addr_state\":0}}"}}
{"add":{"path":"part-00004-7ffc6eab-6c5b-4dc6-a457-4f84a6db7984-c000.snappy.parquet","partitionValues":{},"size":1708,"modificationTime":1584034841000,"dataChange":true,"stats":"{\"numRecords\":31,\"minValues\":{\"loan_id\":16054,\"funded_amnt\":5071,\"paid_amnt\":4047.2439076738106,\"addr_state\":\"CA\"},\"maxValues\":{\"loan_id\":16534,\"funded_amnt\":9948,\"paid_amnt\":9734.894494253329,\"addr_state\":\"WA\"},\"nullCount\":{\"loan_id\":0,\"funded_amnt\":0,\"paid_amnt\":0,\"addr_state\":0}}"}}
{"add":{"path":"part-00005-3eeb59df-59b4-4f99-9d9d-4cc76dc6bf12-c000.snappy.parquet","partitionValues":{},"size":1705,"modificationTime":1584034841000,"dataChange":true,"stats":"{\"numRecords\":31,\"minValues\":{\"loan_id\":16055,\"funded_amnt\":5144,\"paid_amnt\":3706.4731485947177,\"addr_state\":\"CA\"},\"maxValues\":{\"loan_id\":16535,\"funded_amnt\":9958,\"paid_amnt\":9579.188166315795,\"addr_state\":\"WA\"},\"nullCount\":{\"loan_id\":0,\"funded_amnt\":0,\"paid_amnt\":0,\"addr_state\":0}}"}}
{"add":{"path":"part-00006-b0a838a9-9e82-4211-8405-9867a87e1563-c000.snappy.parquet","partitionValues":{},"size":1707,"modificationTime":1584034841000,"dataChange":true,"stats":"{\"numRecords\":31,\"minValues\":{\"loan_id\":16056,\"funded_amnt\":5068,\"paid_amnt\":3484.1474944533356,\"addr_state\":\"CA\"},\"maxValues\":{\"loan_id\":16536,\"funded_amnt\":9947,\"paid_amnt\":9026.019454037367,\"addr_state\":\"WA\"},\"nullCount\":{\"loan_id\":0,\"funded_amnt\":0,\"paid_amnt\":0,\"addr_state\":0}}"}}
{"add":{"path"

In [50]:
%sql
describe history delta.`/tmp/sais_eu_19_demo/loans_delta`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics
31,2020-03-12T17:41:11.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 05928668-07f5-4bde-bd41-b7fe48662d5f, epochId -> 17)",null,List(5918501),0127-045215-pined152,30,WriteSerializable,true,null
30,2020-03-12T17:41:03.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 05928668-07f5-4bde-bd41-b7fe48662d5f, epochId -> 16)",null,List(5918501),0127-045215-pined152,28,WriteSerializable,true,null
29,2020-03-12T17:41:02.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 7dd716af-b214-4990-a725-a2a5ba37f230, epochId -> 12)",null,List(5918501),0127-045215-pined152,28,WriteSerializable,true,null
28,2020-03-12T17:40:54.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 05928668-07f5-4bde-bd41-b7fe48662d5f, epochId -> 15)",null,List(5918501),0127-045215-pined152,26,WriteSerializable,true,null
27,2020-03-12T17:40:51.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 7dd716af-b214-4990-a725-a2a5ba37f230, epochId -> 11)",null,List(5918501),0127-045215-pined152,26,WriteSerializable,true,null
26,2020-03-12T17:40:43.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 05928668-07f5-4bde-bd41-b7fe48662d5f, epochId -> 14)",null,List(5918501),0127-045215-pined152,24,WriteSerializable,true,null
25,2020-03-12T17:40:41.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 7dd716af-b214-4990-a725-a2a5ba37f230, epochId -> 10)",null,List(5918501),0127-045215-pined152,24,WriteSerializable,true,null
24,2020-03-12T17:40:33.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 05928668-07f5-4bde-bd41-b7fe48662d5f, epochId -> 13)",null,List(5918501),0127-045215-pined152,22,WriteSerializable,true,null
23,2020-03-12T17:40:31.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 7dd716af-b214-4990-a725-a2a5ba37f230, epochId -> 9)",null,List(5918501),0127-045215-pined152,22,WriteSerializable,true,null
22,2020-03-12T17:40:24.000+0000,100599,denny.lee@databricks.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 7dd716af-b214-4990-a725-a2a5ba37f230, epochId -> 8)",null,List(5918501),0127-045215-pined152,20,WriteSerializable,true,null


In [51]:
%python
(spark.read.format("delta") \
  .option("versionAsOf", 0) \
  .load(delta_path)
  .count())

Out[34]: 14705

**We see the same number of fully paid loans that we had seen before delete.**

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Rollback
With Delta Lake’s time travel feature allows you to rollback to a previous versions of the table.

You can query by using either a timestamp or a version number using Python, Scala, and/or SQL syntax. For this examples we will query a specific version using the Python syntax.  For more information, refer to [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html) and the [docs](https://docs.delta.io/latest/delta-batch.html#deltatimetravel).

**Let's query the table and get the fully paid loans back**

In [55]:
# Count before rollback
spark.sql("SELECT COUNT(1) FROM loans_delta").show()

In [56]:
# Count of previous Version
spark.sql("SELECT COUNT(*) FROM loans_delta_pre_delete").show()

In [57]:
%scala
# Rollback
spark.read.format("delta")
  .option("versionAsOf", previousVersion) 
  .load(delta_path) 
  .write.format("delta") 
  .mode("overwrite") 
  .save(delta_path)

In [58]:
%python
# Rollback
spark.read.format("delta") \
  .option("versionAsOf", previousVersion) \
  .load(delta_path) \
  .write.format("delta") \
  .mode("overwrite") \
  .save(delta_path)

In [59]:
%python
(spark.read.format("delta") \
  .option("versionAsOf", 19) \
  .load(delta_path)
  .count())

Out[32]: 22455

In [60]:
# Count after rollback
spark.sql("SELECT COUNT(1) FROM loans_delta").show()

In [61]:
# Deleted data is back
spark.sql("SELECT COUNT(1) FROM loans_delta WHERE funded_amnt = paid_amnt").show()

In [62]:
deltaTable.history().show()

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Vacuum old versions of Delta Lake tables

While it's nice to be able to time travel to any previous version, sometimes you want actually delete the data from storage completely for reducing storage costs or for compliance reasons (example, GDPR).
The Vacuum operation deletes data files that have been removed from the table for a certain amount of time. For more information, check out the [docs](https://docs.delta.io/latest/delta-utility.html#vacuum).

By default, `vacuum()` retains all the data needed for the last 7 days. For this example, since this table does not have 7 days worth of history, we will retain 0 hours, which means to only keep the latest state of the table.

In [65]:
%sh 
ls /dbfs/tmp/sais_eu_19_demo/loans_delta/*.parquet | wc -l

In [66]:
spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false")
deltaTable.vacuum(retentionHours = 0)

In [67]:
%sh 
ls /dbfs/tmp/sais_eu_19_demo/loans_delta/*.parquet | wc -l

**Same query as before, but it now fails**

In [69]:
spark.read.format("delta").option("versionAsOf", previousVersion).load(delta_path).createOrReplaceTempView("loans_delta_pre_delete")
spark.sql("SELECT COUNT(*) FROM loans_delta_pre_delete WHERE funded_amnt = paid_amnt").show()

##  ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Upsert into Delta Lake table using Merge
You can upsert data from an Apache Spark DataFrame into a Delta Lake table using the merge operation. This operation is similar to the SQL MERGE command but has additional support for deletes and extra conditions in updates, inserts, and deletes. For more information checkout the [docs](https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-merge).

#### Upsert with Parquet: 7-step process

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

![](https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif)


#### Upsert using with Delta Lake

1-step process: 
1. [Use `Merge` operation](https://docs.delta.io/latest/delta-update.html#upsert-into-a-table-using-merge)

In [72]:
%fs ls /tmp/sais_eu_19_demo/

In [73]:
spark.read.format("parquet").load(parquet_path).where("loan_id < 3").show()

In [74]:
%scala
spark.sql("set spark.sql.shuffle.partitions = 1")
spark.sql("set spark.databricks.delta.snapshotPartitions = 1")

// Configure Delta Lake Silver Path
val deltaSmallPath = "/sais_eu_19_demo/loans_delta_small"

// Remove folder if it exists
println("Deleting directory " + deltaSmallPath)
dbutils.fs.rm(deltaSmallPath, recurse=true)

// Create the Delta table with the same loans data
spark.read.format("parquet").load(parquetPath)
  .where("loan_id < 3")
  .write.format("delta").save(deltaSmallPath)
println("Created a Delta table at " + deltaSmallPath)

spark.read.format("delta").load(deltaSmallPath).createOrReplaceTempView("loans_delta_small")
println("Defined view 'loans_delta_small'")

In [75]:
%python
spark.sql("set spark.sql.shuffle.partitions = 1")
spark.sql("set spark.databricks.delta.snapshotPartitions = 1")

# Configure Delta Lake Silver Path
delta_small_path = "/tmp/sais_eu_19_demo/loans_delta_small"

# Remove folder if it exists
print("Deleting directory " + delta_small_path)
dbutils.fs.rm(delta_small_path, recurse=True)

# Create the Delta table with the same loans data
spark.read.format("parquet").load(parquet_path) \
  .where("loan_id < 3") \
  .write.format("delta").save(delta_small_path)
print("Created a Delta table at " + delta_small_path)

spark.read.format("delta").load(delta_small_path).createOrReplaceTempView("loans_delta_small")
print("Defined view 'loans_delta_small'")

#### Let's focus only on a part of the loans_delta table

In [77]:
spark.sql("select * from loans_delta_small order by loan_id").show()

**Now, let's say we got some new loan information**
1. Duplicate loan_id = 1 was added to the change table due to a delay in processing
1. Existing loan_id = 2 has been fully repaid. The corresponding row needs to be updated.
1. New loan_id = 3 has been funded in CA. This is need to be inserted as a new row.

In [79]:
%scala
val loanUpdates = Seq(
  (1, 1000, 361.19, "WA"), // duplicate information    
  (2, 1000, 1000.0, "TX"), // existing loan's paid_amnt updated, loan paid in full
  (3, 2000, 0.0, "CA"))    // new loan details
  .toDF("loan_id", "funded_amnt", "paid_amnt", "addr_state")

loanUpdates.show()

In [80]:
%python
cols = ['loan_id', 'funded_amnt', 'paid_amnt', 'addr_state']
items = [
  (1, 1000, 361.19, 'WA'), # duplicate information  
  (2, 1000, 1000.0, 'TX'), # existing loan's paid_amnt updated, loan paid in full
  (3, 2000, 0.0, 'CA')     # new loan details
]

loan_updates = spark.createDataFrame(items, cols)

loan_updates.show()

**Merge can upsert this in a single atomic operation.**

SQL `MERGE` command can do both `UPDATE` and `INSERT`.

```

MERGE INTO target t
USING source s
WHEN MATCHED THEN UPDATE SET ...
WHEN NOT MATCHED THEN INSERT ....
```

Since Apache Spark's SQL parser does not have support for parsing MERGE SQL command, we have provided programmatic APIs in Python to perform the same operation with the same semantics as the SQL command.

In [82]:
%scala
import io.delta.tables.DeltaTable
val deltaTable = DeltaTable.forPath(spark, deltaSmallPath)

deltaTable.alias("t").merge(
  loanUpdates.alias("s"), 
  "t.loan_id = s.loan_id")
  .whenMatched.updateAll()
  .whenNotMatched.insertAll()
  .execute()

In [83]:
%python
from delta.tables import *

delta_table = DeltaTable.forPath(spark, delta_small_path)

(delta_table.alias("t").merge(
    loan_updates.alias("s"), 
    "t.loan_id = s.loan_id") 
  .whenMatchedUpdateAll() 
  .whenNotMatchedInsertAll() 
  .execute())

In [84]:
spark.sql("select * from loans_delta_small order by loan_id").show()

**Note the changes in the table**
- Existing loan_id = 2 should have been updated with paid_amnt set to 1000. 
- New loan_id = 3 have been inserted.

<img src="https://docs.delta.io/latest/_static/delta-lake-logo.png" width=300/>
<br/>
## Tutorial Summary

#### Full support for batch and streaming workloads
* Delta Lake allows batch and streaming workloads to concurrently read and write to Delta Lake tables with full ACID transactional guarantees.

#### Schema enforcement and schema evolution
* Delta Lake provides the ability to specify your schema and enforce it. This helps ensure that the data types are correct and required columns are present, preventing bad data from causing data corruption.

#### Table History and Time Travel
* Delta Lake transaction log records details about every change made to data providing a full audit trail of the changes. 
* You can query previous snapshots of data enabling developers to access and revert to earlier versions of data for audits, rollbacks or to reproduce experiments.

#### Delete data and Vacuum old versions
* Delete data from tables using a predicate.
* Fully remove data from previous versions using Vaccum to save storage and satisfy compliance requirements.

#### Upsert data using Merge
* Upsert data into tables from batch and streaming workloads
* Use extended merge syntax for advanced usecases like data deduplication, change data capture, SCD type 2 operations, etc.

## Join the community!


* [Delta Lake on GitHub](https://github.com/delta-io/delta)
* [Delta Lake Slack Channel](https://delta-users.slack.com/) ([Registration Link](https://join.slack.com/t/delta-users/shared_invite/enQtNTY1NDg0ODcxOTI1LWJkZGU3ZmQ3MjkzNmY2ZDM0NjNlYjE4MWIzYjg2OWM1OTBmMWIxZTllMjg3ZmJkNjIwZmE1ZTZkMmQ0OTk5ZjA))
* [Public Mailing List](https://groups.google.com/forum/#!forum/delta-users)